# Tutorial 5: Staking and Slashing

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/blockchain-tokens-and-incentives/blob/main/notebooks/tutorial_05_staking_and_slashing.ipynb)

---

> *"Post collateral. Trade honestly. Earn interest. The system does not require your virtue — only your signature on the stake."*
>
> — Brenn Auster, *Self-Enforcing Agreements* (Year 867)

---

## The Staking Mechanism

Tutorials 01-03 established the theory: the Prisoner's Dilemma shows why traders defect, game theory shows when cooperation can survive, and mechanism design shows how to engineer incentive-compatible rules. Tutorial 04 gave us the building blocks: ERC-20 tokens for currency.

Now we implement Auster's core mechanism: **staking and slashing**. Guild members lock up tokens to participate in the trade network. Honest behavior earns rewards. Misbehavior triggers slashing — automatic, mathematical, and independent of any inspector.

The ICP's Network Nervous System takes this further: validators can lock tokens for up to **8 years**, with reward multipliers that scale with lockup duration. An 8-year lockup earns 4x the rewards of a 1-year lockup. The logic: long-term commitment aligns incentives with long-term network health.

## Learning Objectives

In this tutorial, you will:

1. **Build a staking contract** — lock tokens, track epochs, compute rewards
2. **Implement slashing** — detect misbehavior, confiscate stakes, eject repeat offenders
3. **Analyze the staking dataset** — 387 records across 20 epochs and 20 stakers
4. **Model ICP's 8-year lockup** — reward scaling by lockup duration
5. **Verify incentive compatibility** — prove that the mechanism makes honesty dominant

**Technical Concepts:**
- Proof of Stake fundamentals
- Staking, delegation, and lockup periods
- Slashing conditions and penalties
- Reward scaling and incentive alignment

## Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

staking = pd.read_csv(BASE_URL + "staking_rewards.csv")
guild_trades = pd.read_csv(BASE_URL + "guild_trade_ledger.csv")

print(f"Staking rewards: {len(staking)} records")
print(f"Unique stakers: {staking['staker'].nunique()}")
print(f"Epochs: {staking['epoch'].min()}-{staking['epoch'].max()}")
print(f"\nLockup distributions:")
print(staking.groupby('lockup_years')['staker'].nunique())

## Building the Staking Contract

A staking contract manages the lifecycle: deposit → lock → validate → earn/slash → withdraw. Let's build one from scratch.

In [ ]:
class StakingContract:
    """Guild staking contract with lockup-weighted rewards and slashing."""
    
    LOCKUP_MULTIPLIERS = {1: 1.0, 2: 1.5, 4: 2.5, 8: 4.0}
    BASE_REWARD_RATE = 0.05  # 5% base per epoch
    SLASH_RATES = {
        'late_attestation': 0.0,     # Warning only
        'missed_block': 0.0,          # Reward penalty only
        'double_signing': 0.10,       # 10% of stake
        'equivocation': 0.33,         # 33% of stake
    }
    REWARD_PENALTIES = {
        'late_attestation': 0.10,     # 10% reward reduction
        'missed_block': 0.30,         # 30% reward reduction
        'double_signing': 1.0,        # No reward
        'equivocation': 1.0,          # No reward
    }
    MAX_VIOLATIONS = 3  # Ejection threshold
    
    def __init__(self):
        self.stakers = {}  # address → {stake, lockup, violations, epoch_joined}
        self.epoch = 0
        self.history = []
        self.total_staked = 0
        self.total_slashed = 0
        self.total_rewards = 0
    
    def stake(self, address, amount, lockup_years):
        assert lockup_years in self.LOCKUP_MULTIPLIERS, f"Invalid lockup: {lockup_years}"
        assert address not in self.stakers, f"{address} already staking"
        self.stakers[address] = {
            'stake': amount, 'lockup': lockup_years,
            'violations': 0, 'epoch_joined': self.epoch,
            'status': 'active'
        }
        self.total_staked += amount
    
    def process_epoch(self, behaviors):
        """Process one epoch. behaviors: {address: behavior_type}"""
        self.epoch += 1
        epoch_records = []
        
        for address, info in list(self.stakers.items()):
            if info['status'] == 'ejected':
                continue
            
            behavior = behaviors.get(address, 'honest')
            multiplier = self.LOCKUP_MULTIPLIERS[info['lockup']]
            base_reward = info['stake'] * self.BASE_REWARD_RATE * multiplier
            
            # Apply reward penalty
            reward_penalty = self.REWARD_PENALTIES.get(behavior, 0) * base_reward
            reward = base_reward - reward_penalty
            
            # Apply slashing
            slash_rate = self.SLASH_RATES.get(behavior, 0)
            slash_amount = info['stake'] * slash_rate
            
            if slash_amount > 0:
                info['violations'] += 1
                self.total_slashed += slash_amount
            
            # Update stake
            net = reward - slash_amount
            info['stake'] += net
            self.total_rewards += max(reward, 0)
            
            # Check ejection
            if info['violations'] >= self.MAX_VIOLATIONS or info['stake'] <= 0:
                info['status'] = 'ejected'
                status = 'ejected'
            elif slash_amount > 0:
                status = 'slashed'
            elif reward_penalty > 0:
                status = 'penalized'
            else:
                status = 'active'
            
            epoch_records.append({
                'epoch': self.epoch, 'staker': address,
                'stake': round(info['stake'], 2), 'lockup': info['lockup'],
                'behavior': behavior, 'reward': round(reward, 2),
                'slash': round(slash_amount, 2), 'net': round(net, 2),
                'violations': info['violations'], 'status': status,
                'multiplier': multiplier
            })
        
        self.history.extend(epoch_records)
        return epoch_records
    
    def get_history(self):
        return pd.DataFrame(self.history)

print("Staking contract deployed.")
print(f"  Lockup multipliers: {StakingContract.LOCKUP_MULTIPLIERS}")
print(f"  Slash rates: {StakingContract.SLASH_RATES}")
print(f"  Ejection threshold: {StakingContract.MAX_VIOLATIONS} violations")

In [ ]:
# Simulate 20 epochs with 10 stakers
np.random.seed(867)
contract = StakingContract()

sim_stakers = [
    ('Brenn Auster', 5000, 8),
    ('Torren Gael', 3000, 4),
    ('Tessyn Mord', 2000, 2),
    ('Kellis Vorn', 4000, 8),
    ('Mollen Vek', 1500, 1),
    ('Serath Kyne', 2500, 4),
    ('Vagabu Olt', 1000, 1),
    ('Quonxy', 3500, 2),
    ('Dessa Morin', 2000, 4),
    ('Pemlik Tross', 1800, 1),
]

for name, amount, lockup in sim_stakers:
    contract.stake(name, amount, lockup)

# Run 20 epochs
behaviors_pool = ['honest'] * 15 + ['late_attestation'] * 3 + ['missed_block'] * 2 + \
                 ['double_signing'] * 1 + ['equivocation'] * 1

for epoch in range(20):
    behaviors = {}
    for name, _, _ in sim_stakers:
        if contract.stakers[name]['status'] != 'ejected':
            behaviors[name] = np.random.choice(behaviors_pool)
    contract.process_epoch(behaviors)

sim_history = contract.get_history()
print(f"Simulation complete: {len(sim_history)} records across {contract.epoch} epochs")
print(f"Total rewards distributed: {contract.total_rewards:,.2f}")
print(f"Total slashed: {contract.total_slashed:,.2f}")

## Analyzing the Staking Dataset

Now let's analyze the pre-generated `staking_rewards.csv` dataset — 387 records across 20 epochs.

In [ ]:
# Overview of the staking dataset
print("Staking dataset overview:")
print(f"  Records: {len(staking)}")
print(f"  Stakers: {staking['staker'].nunique()}")
print(f"  Epochs: {staking['epoch'].nunique()}")

print(f"\nBehavior distribution:")
behavior_counts = staking['behavior'].value_counts()
for behavior, count in behavior_counts.items():
    print(f"  {behavior:20s} {count:4d} ({count/len(staking):.1%})")

print(f"\nStatus distribution:")
status_counts = staking['status'].value_counts()
for status, count in status_counts.items():
    print(f"  {status:15s} {count:4d}")

ejected = staking[staking['status'] == 'ejected']['staker'].unique()
print(f"\nEjected stakers: {len(ejected)} — {list(ejected)}")

In [ ]:
# Stake growth over time by lockup duration
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
lockups = [1, 2, 4, 8]

for ax, lockup in zip(axes.flat, lockups):
    subset = staking[staking['lockup_years'] == lockup]
    for staker in subset['staker'].unique():
        s_data = subset[subset['staker'] == staker]
        ax.plot(s_data['epoch'], s_data['stake_balance'], 'o-', markersize=3,
                label=staker[:12], alpha=0.7)
    ax.set_title(f'{lockup}-Year Lockup (multiplier: {StakingContract.LOCKUP_MULTIPLIERS[lockup]}x)')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Stake Balance')
    ax.legend(fontsize=7, loc='upper left')
    ax.grid(alpha=0.3)

plt.suptitle('Stake Growth by Lockup Duration', fontsize=13)
plt.tight_layout()
plt.show()

## The ICP Lockup Model

> *"Eight years. That is what the ICP demands of its most committed validators. In return, they earn four times the base reward. The logic is elegant: an 8-year commitment makes short-term extraction irrational."*
>
> — From the Williams transcript on the Internet Computer

Let's analyze how lockup duration affects total returns.

In [ ]:
# Lockup duration analysis
lockup_stats = staking.groupby('lockup_years').agg(
    stakers=('staker', 'nunique'),
    avg_reward=('base_reward', 'mean'),
    total_rewards=('base_reward', 'sum'),
    total_slashed=('slash_amount', 'sum'),
    avg_yield=('annual_yield_pct', 'first'),
    ejections=('status', lambda x: (x == 'ejected').sum()),
    avg_final_stake=('stake_balance', 'last')
).reset_index()

print("Lockup Duration Analysis:")
print(f"{'Lockup':>8s} {'Stakers':>8s} {'Multiplier':>11s} {'Yield%':>7s} "
      f"{'Avg Reward':>11s} {'Total Slash':>12s} {'Ejections':>10s}")
for _, row in lockup_stats.iterrows():
    mult = StakingContract.LOCKUP_MULTIPLIERS[int(row['lockup_years'])]
    print(f"{int(row['lockup_years']):>6d}yr {int(row['stakers']):>8d} {mult:>11.1f}x "
          f"{row['avg_yield']:>6.0f}% {row['avg_reward']:>11.2f} "
          f"{row['total_slashed']:>12.2f} {int(row['ejections']):>10d}")

In [ ]:
# Compound growth comparison: $1000 staked at each lockup duration
initial = 1000
epochs = 20
growth_curves = {}

for lockup, mult in StakingContract.LOCKUP_MULTIPLIERS.items():
    balance = initial
    curve = [balance]
    for _ in range(epochs):
        reward = balance * StakingContract.BASE_REWARD_RATE * mult
        balance += reward
        curve.append(balance)
    growth_curves[f'{lockup}yr (×{mult})'] = curve

plt.figure(figsize=(10, 6))
colors = ['#e74c3c', '#f39c12', '#2ecc71', '#3498db']
for (label, curve), color in zip(growth_curves.items(), colors):
    plt.plot(range(epochs + 1), curve, 'o-', label=label, color=color, linewidth=2, markersize=4)

plt.xlabel('Epoch')
plt.ylabel('Stake Balance')
plt.title('Compound Growth by Lockup Duration (Starting Stake: 1,000)')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"After {epochs} epochs with no slashing:")
for label, curve in growth_curves.items():
    roi = (curve[-1] - initial) / initial * 100
    print(f"  {label:15s}: {curve[-1]:8,.0f} ({roi:+.0f}% return)")

## Slashing Analysis

Slashing is the enforcement mechanism. Let's analyze which behaviors trigger slashing and what happens to repeat offenders.

In [ ]:
# Slashing events
slashed = staking[staking['slash_amount'] > 0].copy()
print(f"Slashing events: {len(slashed)} out of {len(staking)} records ({len(slashed)/len(staking):.1%})")

print(f"\nSlashing by behavior:")
slash_by_behavior = slashed.groupby('behavior').agg(
    events=('epoch', 'count'),
    avg_slash=('slash_amount', 'mean'),
    total_slash=('slash_amount', 'sum'),
    unique_stakers=('staker', 'nunique')
)
for behavior, row in slash_by_behavior.iterrows():
    print(f"  {behavior:20s}: {int(row['events'])} events, "
          f"avg={row['avg_slash']:.1f}, total={row['total_slash']:.1f}, "
          f"stakers={int(row['unique_stakers'])}")

# Cumulative violations leading to ejection
final_states = staking.groupby('staker').last()
print(f"\nStaker final states:")
print(final_states['status'].value_counts())

In [ ]:
# Track violation accumulation for ejected stakers
ejected_stakers = staking[staking['staker'].isin(ejected)]

if len(ejected_stakers) > 0:
    fig, ax = plt.subplots(figsize=(10, 5))
    for staker in ejected:
        s_data = ejected_stakers[ejected_stakers['staker'] == staker]
        ax.plot(s_data['epoch'], s_data['cumulative_violations'], 'o-',
                label=staker, linewidth=2)
    
    ax.axhline(y=3, color='firebrick', linestyle='--', linewidth=2, label='Ejection threshold')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Cumulative Violations')
    ax.set_title('Violation Accumulation: Ejected Stakers')
    ax.legend()
    ax.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("No stakers were ejected in this dataset.")

## Incentive Compatibility Verification

The critical question: does the staking mechanism make honesty the dominant strategy? Let's verify formally.

In [ ]:
def expected_payoff(stake, lockup, behavior, detection_prob=0.4, epochs=20):
    """Calculate expected total payoff for a given behavior strategy."""
    mult = StakingContract.LOCKUP_MULTIPLIERS[lockup]
    base_rate = StakingContract.BASE_REWARD_RATE
    
    balance = stake
    total_payoff = 0
    violations = 0
    
    for _ in range(epochs):
        if violations >= 3 or balance <= 0:
            break  # Ejected
        
        reward = balance * base_rate * mult
        
        if behavior == 'honest':
            net = reward
        elif behavior == 'occasional_cheat':  # Cheat 20% of the time
            if np.random.random() < 0.2:  # Cheating epoch
                cheat_gain = balance * 0.05  # 5% gain from cheating
                if np.random.random() < detection_prob:
                    slash = balance * 0.10
                    violations += 1
                    net = cheat_gain - slash
                else:
                    net = reward + cheat_gain
            else:
                net = reward
        elif behavior == 'always_cheat':
            cheat_gain = balance * 0.05
            if np.random.random() < detection_prob:
                slash = balance * 0.10
                violations += 1
                net = cheat_gain - slash
            else:
                net = reward + cheat_gain
        else:
            net = reward
        
        balance += net
        total_payoff = balance - stake  # Net gain from initial stake
    
    return total_payoff

# Monte Carlo: compare honest vs cheating strategies
np.random.seed(875)
n_simulations = 1000
strategies = ['honest', 'occasional_cheat', 'always_cheat']
lockups = [1, 2, 4, 8]

results = {}
for lockup in lockups:
    for strategy in strategies:
        payoffs = [expected_payoff(1000, lockup, strategy) for _ in range(n_simulations)]
        results[(lockup, strategy)] = {
            'mean': np.mean(payoffs),
            'std': np.std(payoffs),
            'min': np.min(payoffs),
            'positive_pct': np.mean([p > 0 for p in payoffs])
        }

print(f"Incentive Compatibility Test ({n_simulations} simulations, 20 epochs):\n")
for lockup in lockups:
    print(f"  {lockup}-year lockup:")
    for strategy in strategies:
        r = results[(lockup, strategy)]
        marker = ' ← DOMINANT' if strategy == 'honest' and r['mean'] == max(
            results[(lockup, s)]['mean'] for s in strategies) else ''
        print(f"    {strategy:20s}: mean={r['mean']:8.1f}, "
              f"profit_rate={r['positive_pct']:.0%}{marker}")
    print()

In [ ]:
# Visualize the incentive landscape
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for ax, lockup in zip(axes, lockups):
    means = [results[(lockup, s)]['mean'] for s in strategies]
    stds = [results[(lockup, s)]['std'] for s in strategies]
    colors = ['steelblue', 'darkorange', 'firebrick']
    
    bars = ax.bar(strategies, means, yerr=stds, color=colors, alpha=0.8,
                  edgecolor='black', capsize=5)
    ax.set_title(f'{lockup}-Year Lockup', fontsize=10)
    ax.set_ylabel('Expected Payoff')
    ax.tick_params(axis='x', rotation=30)
    ax.grid(alpha=0.3, axis='y')

plt.suptitle('Expected Payoff by Strategy and Lockup Duration', fontsize=12)
plt.tight_layout()
plt.show()
print("Honest trading dominates across all lockup durations.")

## The Lockup-Risk Tradeoff

Longer lockups earn more but expose stakers to more risk. An 8-year lockup earns 4x rewards but means 8 years of potential slashing events.

In [ ]:
# Risk analysis from the dataset
risk_by_lockup = staking.groupby('lockup_years').agg(
    epochs_active=('epoch', 'count'),
    total_earned=('base_reward', 'sum'),
    total_slashed=('slash_amount', 'sum'),
    max_drawdown=('net_reward', 'min'),
    ejection_count=('status', lambda x: (x == 'ejected').sum()),
    avg_final_balance=('stake_balance', 'last')
).reset_index()

risk_by_lockup['slash_reward_ratio'] = risk_by_lockup['total_slashed'] / risk_by_lockup['total_earned'].clip(lower=1)

print("Risk-Reward Analysis by Lockup Duration:")
print(f"{'Lockup':>8s} {'Active':>7s} {'Earned':>10s} {'Slashed':>10s} "
      f"{'Slash/Earn':>11s} {'Max Loss':>10s}")
for _, row in risk_by_lockup.iterrows():
    print(f"{int(row['lockup_years']):>6d}yr {int(row['epochs_active']):>7d} "
          f"{row['total_earned']:>10.1f} {row['total_slashed']:>10.1f} "
          f"{row['slash_reward_ratio']:>10.2%} {row['max_drawdown']:>10.1f}")

In [ ]:
# Sharpe-ratio-like metric: excess return per unit of risk
# For each staker, compute annualized return and volatility
staker_metrics = []

for staker in staking['staker'].unique():
    s_data = staking[staking['staker'] == staker]
    rewards = s_data['net_reward'].values
    lockup = s_data['lockup_years'].iloc[0]
    initial = s_data['stake_balance'].iloc[0] - s_data['net_reward'].iloc[0]
    final = s_data['stake_balance'].iloc[-1]
    
    total_return = (final - initial) / max(initial, 1)
    volatility = np.std(rewards) / max(np.mean(np.abs(rewards)), 1)
    sharpe = total_return / max(volatility, 0.01)
    
    staker_metrics.append({
        'staker': staker, 'lockup': lockup,
        'total_return': total_return, 'volatility': volatility,
        'sharpe': sharpe, 'final_status': s_data['status'].iloc[-1]
    })

metrics_df = pd.DataFrame(staker_metrics)

plt.figure(figsize=(10, 6))
for lockup in [1, 2, 4, 8]:
    subset = metrics_df[metrics_df['lockup'] == lockup]
    colors = ['firebrick' if s == 'ejected' else 'steelblue' for s in subset['final_status']]
    plt.scatter(subset['volatility'], subset['total_return'], 
                c=colors, s=100, alpha=0.7, edgecolors='black',
                label=f'{lockup}yr lockup')
    for _, row in subset.iterrows():
        plt.annotate(row['staker'][:8], (row['volatility'], row['total_return']),
                     fontsize=7, alpha=0.7)

plt.xlabel('Reward Volatility (risk)')
plt.ylabel('Total Return')
plt.title('Risk-Return Profile by Staker (blue=active, red=ejected)')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## The Complete Staking Lifecycle

Let's trace one staker's full journey through the mechanism.

In [ ]:
# Pick a staker with interesting history (some violations but not ejected)
interesting = staking[
    (staking['cumulative_violations'] > 0) & 
    (staking['status'] != 'ejected')
]['staker'].unique()

if len(interesting) > 0:
    focus_staker = interesting[0]
else:
    focus_staker = staking['staker'].iloc[0]

s_data = staking[staking['staker'] == focus_staker].copy()

print(f"=== Staking Lifecycle: {focus_staker} ===")
print(f"Lockup: {s_data['lockup_years'].iloc[0]} years")
print(f"Epochs active: {len(s_data)}")
print(f"Final status: {s_data['status'].iloc[-1]}")
print(f"\nEpoch-by-epoch:")
print(f"{'Epoch':>5s} {'Balance':>10s} {'Reward':>8s} {'Slash':>8s} {'Net':>8s} {'Behavior':>18s} {'Status':>10s}")
for _, row in s_data.iterrows():
    print(f"{int(row['epoch']):5d} {row['stake_balance']:10.1f} {row['base_reward']:8.1f} "
          f"{row['slash_amount']:8.1f} {row['net_reward']:8.1f} "
          f"{row['behavior']:>18s} {row['status']:>10s}")

## Exercises

### Exercise 1: Optimal Lockup Duration

Given the slashing rates in the dataset, what is the optimal lockup duration? Consider both expected return and risk of ejection.

In [ ]:
# Compare risk-adjusted returns by lockup
lockup_returns = metrics_df.groupby('lockup').agg(
    avg_return=('total_return', 'mean'),
    avg_sharpe=('sharpe', 'mean'),
    ejection_rate=('final_status', lambda x: (x == 'ejected').mean())
).sort_values('avg_sharpe', ascending=False)

print("Lockup Duration Comparison:")
print(f"{'Lockup':>8s} {'Avg Return':>11s} {'Sharpe':>8s} {'Ejection%':>10s}")
for lockup, row in lockup_returns.iterrows():
    marker = ' ← OPTIMAL' if lockup == lockup_returns.index[0] else ''
    print(f"{lockup:>6d}yr {row['avg_return']:>10.2%} {row['avg_sharpe']:>8.2f} "
          f"{row['ejection_rate']:>10.0%}{marker}")

### Exercise 2: Slashing Sensitivity

How much can we reduce the double_signing slash rate before cheating becomes profitable? Vary the slash rate from 1% to 50% and find the break-even point.

In [ ]:
# Slashing sensitivity analysis
np.random.seed(42)
slash_rates = np.linspace(0.01, 0.50, 25)
honest_advantage = []

for sr in slash_rates:
    honest_payoffs = []
    cheat_payoffs = []
    for _ in range(200):
        # Honest
        bal = 1000
        for _ in range(20):
            bal += bal * 0.05 * 2.5  # 4-year lockup
        honest_payoffs.append(bal - 1000)
        
        # Cheater (20% cheat rate, detection prob 0.4)
        bal = 1000
        violations = 0
        for _ in range(20):
            if violations >= 3 or bal <= 0:
                break
            if np.random.random() < 0.2:  # Cheat
                if np.random.random() < 0.4:  # Detected
                    bal -= bal * sr
                    violations += 1
                else:
                    bal += bal * 0.05 * 2.5 + bal * 0.05
            else:
                bal += bal * 0.05 * 2.5
        cheat_payoffs.append(bal - 1000)
    
    honest_advantage.append(np.mean(honest_payoffs) - np.mean(cheat_payoffs))

plt.figure(figsize=(10, 5))
plt.plot(slash_rates * 100, honest_advantage, 'o-', color='steelblue', linewidth=2)
plt.axhline(y=0, color='firebrick', linestyle='--', linewidth=2)
plt.xlabel('Double-Signing Slash Rate (%)')
plt.ylabel('Honest Advantage (positive = honesty wins)')
plt.title('Slashing Sensitivity: When Does Cheating Become Profitable?')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

for sr, adv in zip(slash_rates, honest_advantage):
    if adv <= 0:
        print(f"Break-even: slash rate ≈ {sr:.0%} (below this, cheating pays)")
        break
else:
    print("Honesty dominates across all tested slash rates.")

### Exercise 3: Delegation

Not everyone wants to run a validator. Delegation lets small holders stake through a validator. Implement a simple delegation model where delegators earn rewards minus a commission.

In [ ]:
class DelegatedStaking:
    """Staking with delegation support."""
    def __init__(self):
        self.validators = {}  # validator → {own_stake, commission_rate}
        self.delegations = defaultdict(dict)  # validator → {delegator: amount}
    
    def register_validator(self, name, stake, commission_rate=0.10):
        self.validators[name] = {'stake': stake, 'commission': commission_rate}
    
    def delegate(self, delegator, validator, amount):
        assert validator in self.validators
        self.delegations[validator][delegator] = \
            self.delegations[validator].get(delegator, 0) + amount
    
    def distribute_rewards(self, validator, total_reward):
        """Distribute epoch reward to validator and delegators."""
        v_info = self.validators[validator]
        total_delegated = sum(self.delegations[validator].values())
        total_stake = v_info['stake'] + total_delegated
        
        # Validator commission on delegated rewards
        delegated_share = total_reward * (total_delegated / total_stake)
        commission = delegated_share * v_info['commission']
        
        # Validator gets: own share + commission
        v_own_share = total_reward * (v_info['stake'] / total_stake)
        v_total = v_own_share + commission
        
        distributions = [{'recipient': validator, 'amount': round(v_total, 2),
                          'type': 'validator'}]
        
        # Delegators get: their share - commission
        for delegator, del_amount in self.delegations[validator].items():
            share = total_reward * (del_amount / total_stake)
            del_commission = share * v_info['commission']
            net = share - del_commission
            distributions.append({'recipient': delegator, 'amount': round(net, 2),
                                  'type': 'delegator'})
        
        return distributions

# Demo: Brenn Auster runs a validator, 3 scholars delegate
ds = DelegatedStaking()
ds.register_validator('Brenn Auster', 5000, commission_rate=0.10)
ds.delegate('Vagabu Olt', 'Brenn Auster', 500)
ds.delegate('Dessa Morin', 'Brenn Auster', 1000)
ds.delegate('Pemlik Tross', 'Brenn Auster', 750)

epoch_reward = 500  # Total reward for this validator's pool
distributions = ds.distribute_rewards('Brenn Auster', epoch_reward)

print(f"Delegation Reward Distribution (total reward: {epoch_reward}):")
for d in distributions:
    print(f"  {d['recipient']:15s} ({d['type']:9s}): {d['amount']:8.2f}")
print(f"\n  Sum: {sum(d['amount'] for d in distributions):.2f}")

## Summary

In this tutorial, we learned:

1. **Staking** locks tokens as collateral — honest validators earn rewards, dishonest ones lose stakes
2. **Lockup multipliers** (ICP model) reward long-term commitment: 8-year lockup = 4x rewards
3. **Slashing** is the enforcement mechanism — double-signing costs 10% of stake, equivocation costs 33%
4. **Ejection** removes repeat offenders after 3 violations
5. **Incentive compatibility** is verified: honest behavior is the dominant strategy across all lockup durations

**Key insight:** Staking transforms the Prisoner's Dilemma. Without staking, defection is dominant. With staking, cooperation is dominant — not because the game changed, but because the payoff matrix changed. The stake makes the cost of defection exceed its gain, automatically and without inspectors.

This is Auster's complete answer to the guild's problem from Tutorial 01.

---

**Next Tutorial:** Liquidity and AMMs — automated market makers for trustless commodity exchange.

---

> *"The Capital tried rules. The Guild tried honor. Both failed. I tried mathematics — and discovered that honesty is just the cheapest strategy."*
>
> — Brenn Auster